In [1]:
# Core libraries for data handling, date operations, and finance API access
import numpy as np
import yfinance as yf
from datetime import datetime, timedelta
import math
import json
import os
from typing import Dict, Any, Tuple, Optional

In [2]:
def print_line(symbol: str = "-", length: int = 70) -> None:
    """Print a line of '-' (or symbol) repeated length times."""
    print(symbol * length)

def print_header(title: str) -> None:
    """Surround title with two lines of default separators."""
    print_line()
    print(title)
    print_line()

def compute_profit_metrics(starting_capital: float, final_capital: float) -> Tuple[float, float, float]:
    """
    Compute net profit and percentage change from starting to final capital,
    print a summary, and return (final_capital, profit, profit_percent).
    """
    profit = final_capital - starting_capital
    profit_percent = (profit / starting_capital) * 100
    plus_minus = "+" if profit >= 0 else ""

    print(f"\nInitial Capital: ${starting_capital:,.2f}")
    print(f"Final Capital ${final_capital:,.2f}")
    print(f"Capital Increase: {plus_minus}${profit:,.2f} ({plus_minus}{profit_percent:.2f}%)\n")

    return final_capital, profit, profit_percent


In [3]:
def fetch_stock_price(ticker: str, date: str) -> float:
    """
    Fetches the stock price for the given ticker from Yahoo Finance on a specific date.
    """
    try:
        # Fetch the stock data
        stock = yf.Ticker(ticker)


        # Get the stock history around the specified date
        stock_info = stock.history(start=date, end="2024-12-15")

        if not stock_info.empty:
            price_on_date = stock_info['Close'].iloc[0]
            return round(price_on_date, 2)
        else:
            raise ValueError(f"No data returned for {ticker} on {date}.")
    except Exception as e:
        print(f"Error fetching stock price for {ticker} on {date}: {e}")
        return None

In [4]:
def get_first_last_dates(stocks_data: Dict[str, Dict[str, Dict[str, list]]]) -> Tuple[Optional[str], Optional[str]]:
    """
    Return earliest and latest dates (YYYY-MM-DD) in stocks_data, or (None, None) if no dates found.
    """
    all_dates = []
    # Collect all date strings from each group's 'Dates' list
    for groups in stocks_data.values():
        if isinstance(groups, dict):
            for group_data in groups.values():
                if isinstance(group_data, dict) and 'Dates' in group_data:
                    all_dates.extend(group_data['Dates'])
    
    if not all_dates:
        return None, None
    
    # Parse into datetime objects, ignoring invalid formats
    date_objs = []
    for date_str in all_dates:
        try:
            date_objs.append(datetime.strptime(date_str, '%Y-%m-%d'))
        except ValueError as e:
            print(f"Error parsing date '{date_str}': {e}")
    
    if not date_objs:
        return None, None
    
    # Determine earliest and latest
    first_dt = min(date_objs)
    last_dt  = max(date_objs)
    
    return first_dt.strftime('%Y-%m-%d'), last_dt.strftime('%Y-%m-%d')


In [5]:
def buy_and_hold_all_stocks(
    stocks_data: Dict[str, Any],
    starting_capital: float = 100_000,
    transaction_cost_rate: float = 0.0002
) -> Tuple[float, float, float]:    
    """
    Simulate a realistic buy-and-hold strategy across all stocks:
    
    1. Equally split `starting_capital` across each ticker.
    2. Deduct buy-side fees and invest only the net amount, leaving real cash residue.
    3. Buy on each stock’s first date and sell on its last, applying fees both ways.
    4. Return overall performance metrics: final capital, net profit, and profit %.
    """
    final_capital_per_stock = {}
    total_number_of_stocks = len(stocks_data)
    starting_capital_per_stock = starting_capital / total_number_of_stocks

    for stock, weeks_data in stocks_data.items():
        print(f"\nProcessing stock: {stock}")

        # Get first and last week's data
        first_week = weeks_data[min(weeks_data.keys(), key=int)]['Dates']
        last_week = weeks_data[max(weeks_data.keys(), key=int)]['Dates']
        buying_day = first_week[0]
        selling_day = last_week[-1]

        # Get first and last day's actual prices
        first_day_buy_price = fetch_stock_price(stock, buying_day)
        last_day_sell_price = fetch_stock_price(stock, selling_day)

        if first_day_buy_price is None or last_day_sell_price is None:
            print(f"Skipping {stock} due to missing price data.")
            continue

        # Buy on the first day
        allocated_capital_after_costs = starting_capital_per_stock * (1 - transaction_cost_rate)
        stock_units = int(math.floor(allocated_capital_after_costs / first_day_buy_price))
        print(f"Bought {stock_units} {stock} stock units on the first day {buying_day} at ${first_day_buy_price:.2f}.")

        # Sell on the last day
        final_capital = stock_units * last_day_sell_price * (1 - transaction_cost_rate)
        print(f"Sold {stock_units} {stock} stock units on the last day {selling_day} at ${last_day_sell_price:.2f}. Final Capital: ${final_capital:.2f}")

        final_capital_per_stock[stock] = final_capital

    final_capital_all_stocks = sum(final_capital_per_stock.values())

    return compute_profit_metrics(starting_capital, final_capital_all_stocks)



In [6]:
def compare_strategies(
    day_trading_results: tuple,
    buy_and_hold_results: tuple,
    dow_jones_index_results: tuple,
    initial_capital: float
) -> None:
    """
    Print a side-by-side performance comparison of three strategies:

    1. Show each strategy’s final capital and percentage return.
    2. Identify the top performer.
    3. Report by how much the best strategy outperformed the others.
    """
    # Unpack the result tuples
    day_final, day_profit, day_pct      = day_trading_results
    hold_final, hold_profit, hold_pct   = buy_and_hold_results
    dj_final, dj_profit, dj_pct         = dow_jones_index_results

    print_header("Performance Evaluation of Investment Strategies...")

    # Initial capital
    print(f"\nInitial Capital: ${initial_capital:,.2f}\n")

    # Aligned summary
    for name, final, profit, pct in [
        ("Trading Bot",     day_final,  day_profit,  day_pct),
        ("Buy and Hold",    hold_final, hold_profit, hold_pct),
        ("Dow Jones Index", dj_final,   dj_profit,   dj_pct),
    ]:
        sign = "+" if pct >= 0 else ""
        # name left-aligned in 20 chars; final in 12 chars; return fixed-width 5 chars
        print(f"{name:<15}  Final Capital: ${final:,.2f}  Return: {sign}{pct:>5.2f}%")

    # Compute outperformance
    returns = {
        "Trading Bot":     day_pct,
        "Buy and Hold":    hold_pct,
        "Dow Jones Index": dj_pct,
    }
    best_name = max(returns, key=returns.get)
    best_pct  = returns[best_name]

    print("\nOutperformance:")
    for name, pct in returns.items():
        if name == best_name:
            continue
        diff = best_pct - pct
        print(f"{best_name:<16} outperformed {name:<15} by {diff:>5.2f}%")


In [7]:
def trading_bot(
    stocks_data: Dict[str, Dict[str, Dict[str, Any]]],
    starting_capital: float = 100_000,
    transaction_cost_rate: float = 0.0002,
    lookout_window: int = 5,
    max_traded_stocks: int = 1,
    gain_threshold: float = 0.02,
    loss_threshold: float = -0.01
) -> Tuple[float, float, float]:
    """
    Execute a weekly trading strategy over predicted stock data:

    1. At the start of each week, score stocks by their average predicted change
       over the next lookout_window days and buy up to max_traded_stocks top picks.
    2. After at least 3 days, sell holdings if gain ≥ gain_threshold or loss ≤ loss_threshold.
    3. Force-sell any remaining positions at week’s end (day 5) to reset for next week.
    4. Apply transaction_cost_rate to all buys and sells.
    5. Print each trade’s details and return final capital, net profit, and profit percent.
    """

    capital = starting_capital

    # Dictionary to track current holdings:
    # Format: holding_stocks[stock_name] = {
    #   'units': number_of_shares_held,
    #   'buy_price': price_per_share_at_purchase,
    #   'date_bought': date_string (YYYY-MM-DD)
    # }
    holding_stocks = {}

    # Maximum amount to invest in each stock, ensuring diversification and risk management
    max_investment_per_stock = starting_capital * (1 / max_traded_stocks)

    # We find how many weeks there are by looking at the first stock's data length
    total_weeks = len(list(stocks_data.values())[0])

    for week_number in range(1, total_weeks + 1):
        print(f"\nProcessing week {week_number}")

        daily_data = []

        # Gather predictions for each stock for each day in this week
        # We want to find the average and max predicted future price change over the next `lookout_window` days
        for stock, weeks_data in stocks_data.items():
            if str(week_number) in weeks_data:
                week_data = weeks_data[str(week_number)]
                dates = week_data['Dates']
                predicted_changes = week_data['predicted_change']

                for i, date in enumerate(dates):
                    # Extract the next `lookout_window` predicted changes
                    future_preds = predicted_changes[i:i + lookout_window]

                    # Only proceed if we have a full window of future predictions
                    if len(future_preds) == lookout_window:
                        avg_future_prediction = np.mean(future_preds)    # Average predicted change
                        max_future_prediction = max(future_preds)       # Maximum predicted change

                        daily_data.append({
                            'stock': stock,
                            'date': date,
                            'avg_future_prediction': avg_future_prediction,
                            'max_future_prediction': max_future_prediction
                        })

        # Sort the daily_data based on the average future prediction, best (highest) first
        daily_data.sort(key=lambda x: x['avg_future_prediction'], reverse=True)

        # Buy logic: If we are not holding any stocks, consider buying top candidates
        # We only buy if we have capacity (not holding anything) and follow the top predicted stocks
        if not holding_stocks:
            top_stocks = daily_data[:max_traded_stocks]
            for stock_data in top_stocks:
                stock = stock_data['stock']
                date = stock_data['date']

                # Fetch the actual price of the stock on 'date' to simulate a real buy
                buy_price = fetch_stock_price(stock, date)

                # Validate we got a price and have capital to invest
                if buy_price and capital > 1:
                    # Invest up to max_investment_per_stock or the entire capital times (1 - cost) if smaller
                    investment_amount = min(max_investment_per_stock, capital * (1 - transaction_cost_rate))
                    units = investment_amount / buy_price

                    capital -= investment_amount

                    holding_stocks[stock] = {
                        'units': units,
                        'buy_price': buy_price,
                        'date_bought': date
                    }

                    print(f"{date}: Bought {stock}. Units: {units:.2f} at ${buy_price:.2f}. Remaining capital: {capital:.2f}")
                else:
                    print(f"Failed to fetch buy price for {stock} on {date} or insufficient capital. Skipping.")

        # Sell logic: Check existing holdings and decide if we should sell
        # We try to sell after at least 3 days, either because we hit profit/loss thresholds or at the week's end.
        to_sell = []

        for stock, details in holding_stocks.items():
            # Determine the sell date: 3 days after buy date (for initial threshold check)
            buy_date = datetime.strptime(details['date_bought'], "%Y-%m-%d")
            threshold_sell_date = (buy_date + timedelta(days=3)).strftime("%Y-%m-%d")

            sell_price = fetch_stock_price(stock, threshold_sell_date)

            # Calculate gain or loss percentage
            if sell_price:
                gain = (sell_price - details['buy_price']) / details['buy_price']
                plus_minus = "+" if gain >= 0 else ""

                print(f"Checking {stock} on {threshold_sell_date}: Buy Price = ${details['buy_price']:.2f}, "
                      f"Sell Price = ${sell_price:.2f}, Gain = {plus_minus}{gain:.4%}")

                # Decide whether to sell based on thresholds
                if gain >= gain_threshold or gain <= loss_threshold:
                    proceeds = details['units'] * sell_price * (1 - transaction_cost_rate)
                    capital += proceeds
                    print(f"{threshold_sell_date}: Sold {stock} at ${sell_price:.2f}. Capital: {capital:.2f}")
                    to_sell.append(stock)
            else:
                print(f"Failed to fetch sell price for {stock} on {threshold_sell_date}. Retaining holdings for now.")

        # Remove sold stocks from holdings
        for stock in to_sell:
            del holding_stocks[stock]

        # At the end of the week, force sell any remaining holdings that were not sold earlier
        # This ensures we do not carry the same holdings into the next week indefinitely.
        to_remove = []

        for stock in list(holding_stocks.keys()):
            details = holding_stocks[stock]
            # The forced sell date: 5 days after the buy date
            forced_sell_date = datetime.strptime(details['date_bought'], "%Y-%m-%d") + timedelta(days=5)
            forced_sell_date_str = forced_sell_date.strftime("%Y-%m-%d")

            sell_price = fetch_stock_price(stock, forced_sell_date_str)

            if sell_price:
                # Sell at forced date regardless of gain/loss
                proceeds = details['units'] * sell_price * (1 - transaction_cost_rate)
                capital += proceeds
                print(f"End of week: Forced sell of {stock} at ${sell_price:.2f}. Capital: {capital:.2f}")
                to_remove.append(stock)
            else:
                print(f"End of week: Failed to fetch price for {stock} on {forced_sell_date_str}. Retaining holdings.")

        # Remove the forcibly sold stocks from holdings
        for stock in to_remove:
            del holding_stocks[stock]

    # Attempt to sell any remaining holdings at the final forced date.
    for stock, details in holding_stocks.items():
        final_sell_date = datetime.strptime(details['date_bought'], "%Y-%m-%d") + timedelta(days=5)
        final_sell_date_str = final_sell_date.strftime("%Y-%m-%d")

        sell_price = fetch_stock_price(stock, final_sell_date_str)

        if sell_price:
            # Sell whatever is left at the final cleanup
            proceeds = details['units'] * sell_price * (1 - transaction_cost_rate)
            capital += proceeds
            print(f"Final Sell: {stock} at ${sell_price:.2f}. Capital: {capital:.2f}")
        else:
            print(f"Failed to fetch final price for {stock}. Retaining holdings. Capital remains ${capital:.2f}")

    return compute_profit_metrics(starting_capital, capital)

In [8]:
def buy_and_hold_index(
    ticker: str,
    stocks_data: Dict[str, Any],
    starting_capital: float = 100_000,
    transaction_cost_rate: float = 0.0002
) -> Tuple[float, float, float]:
    """
    Simulate a buy-and-hold strategy for an index ticker using fractional shares:

    1. Determine the overall date range via get_first_last_dates().
    2. Fetch the index’s closing price on the first and last dates.
    3. Invest the entire starting_capital net of the buy-side fee.
    4. Sell all fractional shares at the end date net of the sell-side fee.
    5. Return (final_capital, profit, profit_percent) via compute_profit_metrics().
    """
    # Fetch only the two required prices
    start_date, end_date = get_first_last_dates(stocks_data)
    first_day_buy_price = fetch_stock_price(ticker, start_date)
    last_day_buy_price  = fetch_stock_price(ticker, end_date)

    if first_day_buy_price is None or last_day_buy_price is None:
        raise ValueError(f"Error fetching price for {ticker} on {start_date} or {end_date}")

    # Deduct buy-side fee and invest all remaining capital fractionally
    capital_after_buy_fee = starting_capital * (1 - transaction_cost_rate)
    stock_units = capital_after_buy_fee / first_day_buy_price

    # On sell, apply sell-side fee to all proceeds
    sale_proceeds = stock_units * last_day_buy_price * (1 - transaction_cost_rate)
    final_capital = sale_proceeds

    return compute_profit_metrics(starting_capital, final_capital)
    

In [9]:
def main():
    """
    Orchestrate end-to-end strategy simulations and comparison:

    1. Load stock prediction data from JSON.
    2. Initialize starting capital.
    3. Run the smart trading bot.
    4. Run buy-and-hold across individual stocks.
    5. Run buy-and-hold for the Dow Jones Index.
    6. Compare and report on all strategies’ performance.
    """
    # 1) Load stock predictions from JSON file
    file_path = 'stock_sample_predictions_52_weeks.json'
    try:
        with open(file_path, 'r') as file:
            stocks_data = json.load(file)
    except FileNotFoundError:
        raise RuntimeError(f"Could not find data file at {file_path}")

    # 2) Set up initial capital for all simulations
    initial_capital = 100_000

    # 3) Run the smart trading bot simulation
    print_header("Running Trading Bot...")
    day_trading_results = trading_bot(
        stocks_data,
        starting_capital=initial_capital,
        max_traded_stocks=2,
        gain_threshold=0.7,
        loss_threshold=-0.07
    )

    # 4) Run the buy-and-hold simulation across all stocks
    print_header("Running Buy and Hold Strategy...")
    buy_and_hold_results = buy_and_hold_all_stocks(
        stocks_data,
        starting_capital=initial_capital
    )

    # 5) Run the buy-and-hold simulation for the Dow Jones Index
    print_header("Running Dow Jones Index Strategy...")
    dow_jones_index_results = buy_and_hold_index(
        '^DJI',
        stocks_data,
        starting_capital=initial_capital
    )

    # 6) Compare and report on the performance of all strategies
    compare_strategies(
        day_trading_results,
        buy_and_hold_results,
        dow_jones_index_results,
        initial_capital
    )

if __name__ == "__main__":
    main()

----------------------------------------------------------------------
Running Trading Bot...
----------------------------------------------------------------------

Processing week 1
2023-11-21: Bought AMZN. Units: 347.46 at $143.90. Remaining capital: 50000.00
2023-11-21: Bought IBM. Units: 341.04 at $146.58. Remaining capital: 10.00
Checking AMZN on 2023-11-24: Buy Price = $143.90, Sell Price = $146.74, Gain = +1.9736%
Checking IBM on 2023-11-24: Buy Price = $146.58, Sell Price = $147.79, Gain = +0.8255%
End of week: Forced sell of AMZN at $147.73. Capital: 51330.52
End of week: Forced sell of IBM at $148.16. Capital: 101849.26

Processing week 2
2023-11-29: Bought TSLA. Units: 204.80 at $244.14. Remaining capital: 51849.26
2023-12-05: Bought SIE.DE. Units: 331.52 at $150.82. Remaining capital: 1849.26
Checking TSLA on 2023-12-02: Buy Price = $244.14, Sell Price = $235.58, Gain = -3.5062%
Checking SIE.DE on 2023-12-08: Buy Price = $150.82, Sell Price = $154.31, Gain = +2.3140%
End o